In [2]:
# Instalación de la librería ultralytics (contiene YOLOv8)
# Esta librería proporciona los modelos pre-entrenados de YOLO versión 8
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.7 MB/s eta 0:00:00a 0:00:01


In [3]:
# Importación de la clase YOLO
# La clase YOLO es la interfaz principal para trabajar con los modelos YOLOv8
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Carga del modelo YOLOv8
# 'yolov8n.pt' es el modelo nano (el más pequeño y rápido)
# Otras opciones: yolov8s.pt (small), yolov8m.pt (medium), yolov8l.pt (large), yolov8x.pt (extra large)
# El modelo se descarga automáticamente la primera vez
model = YOLO('yolov8n.pt')

print("Modelo YOLOv8 cargado exitosamente")
print(f"Número de clases que puede detectar: {len(model.names)}")
print(f"Algunas clases: {list(model.names.values())[:10]}")

Modelo YOLOv8 cargado exitosamente
Número de clases que puede detectar: 80
Algunas clases: ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light']


## 📝 Descripción de cada parte del código

### 1. **Instalación de ultralytics**
- `!pip install ultralytics`: Instala la librería oficial de Ultralytics que contiene YOLOv8
- Esta librería incluye todo lo necesario para detección de objetos, segmentación y clasificación

### 2. **Importaciones**
- `YOLO`: Clase principal para cargar y usar modelos YOLOv8
- `cv2`: OpenCV para procesamiento de imágenes
- `numpy`: Para operaciones con arrays
- `PIL`: Para manipulación de imágenes
- `matplotlib`: Para visualizar resultados

### 3. **Carga del modelo**
- `YOLO('yolov8n.pt')`: Carga el modelo YOLOv8 nano
  - **n** = nano (más rápido, menos preciso) ~3MB
  - **s** = small ~11MB
  - **m** = medium ~25MB
  - **l** = large ~43MB
  - **x** = extra large (más lento, más preciso) ~68MB

### 4. **Información del modelo**
- El modelo puede detectar **80 clases** de objetos del dataset COCO
- Incluye: personas, vehículos, animales, objetos cotidianos, etc.

In [11]:
# Sistema de Detección con Múltiples Modelos

class SistemaDeteccionHibrido:
    """
    Sistema que combina múltiples modelos YOLOv8 para detectar diferentes tipos de objetos
    """
    
    def __init__(self, modelo_coco_path='yolov8n.pt', modelo_armas_path=None):
        """
        Inicializa el sistema con múltiples modelos
        
        Args:
            modelo_coco_path: Ruta al modelo COCO (por defecto yolov8n.pt)
            modelo_armas_path: Ruta al modelo de armas personalizado (opcional)
        """
        print("🔧 Inicializando Sistema Híbrido de Detección...")
        
        # Cargar modelo COCO
        self.modelo_coco = YOLO(modelo_coco_path)
        print(f"✓ Modelo COCO cargado: {modelo_coco_path}")
        
        # Cargar modelo de armas si existe
        self.modelo_armas = None
        if modelo_armas_path and os.path.exists(modelo_armas_path):
            self.modelo_armas = YOLO(modelo_armas_path)
            print(f"✓ Modelo de armas cargado: {modelo_armas_path}")
        else:
            print("⚠️ Modelo de armas no disponible (usar ruta válida)")
        
        # Definir clases peligrosas de COCO
        self.clases_peligrosas_coco = {
            'bottle': '🍾 BOTELLA',
            'knife': '🔪 CUCHILLO',
            'baseball bat': '⚾ BATE',
            'scissors': '✂️ TIJERAS'
        }
        
        print("\n📋 Configuración del sistema:")
        print(f"   - Modelos activos: {1 + (1 if self.modelo_armas else 0)}")
        print(f"   - Clases COCO: {len(self.clases_peligrosas_coco)}")
        if self.modelo_armas:
            print(f"   - Clases armas: {len(self.modelo_armas.names)}")
    
    def detectar(self, frame, conf_threshold=0.5):
        """
        Realiza detección con todos los modelos disponibles
        
        Args:
            frame: Imagen/frame a analizar
            conf_threshold: Umbral de confianza
            
        Returns:
            Lista de detecciones combinadas de todos los modelos
        """
        detecciones = []
        
        # Detección con modelo COCO
        resultados_coco = self.modelo_coco.predict(
            source=frame, 
            conf=conf_threshold, 
            verbose=False
        )
        
        for r in resultados_coco:
            for box in r.boxes:
                clase_id = int(box.cls)
                nombre_clase = self.modelo_coco.names[clase_id]
                
                if nombre_clase in self.clases_peligrosas_coco:
                    detecciones.append({
                        'tipo': 'COCO',
                        'nombre': nombre_clase,
                        'alerta': self.clases_peligrosas_coco[nombre_clase],
                        'confianza': float(box.conf),
                        'bbox': box.xyxy[0].cpu().numpy(),
                        'color': (0, 100, 255)  # Naranja
                    })
        
        # Detección con modelo de armas (si existe)
        if self.modelo_armas:
            resultados_armas = self.modelo_armas.predict(
                source=frame,
                conf=conf_threshold,
                verbose=False
            )
            
            for r in resultados_armas:
                for box in r.boxes:
                    clase_id = int(box.cls)
                    nombre_clase = self.modelo_armas.names[clase_id]
                    
                    detecciones.append({
                        'tipo': 'ARMAS',
                        'nombre': nombre_clase,
                        'alerta': f'🔫 {nombre_clase.upper()}',
                        'confianza': float(box.conf),
                        'bbox': box.xyxy[0].cpu().numpy(),
                        'color': (0, 0, 255)  # Rojo
                    })
        
        return detecciones
    
    def dibujar_detecciones(self, frame, detecciones):
        """
        Dibuja todas las detecciones en el frame
        """
        frame_anotado = frame.copy()
        
        for det in detecciones:
            # Extraer coordenadas
            x1, y1, x2, y2 = map(int, det['bbox'])
            
            # Dibujar caja
            cv2.rectangle(frame_anotado, (x1, y1), (x2, y2), det['color'], 3)
            
            # Preparar texto
            texto = f"{det['alerta']} {det['confianza']:.0%}"
            
            # Fondo del texto
            (w, h), _ = cv2.getTextSize(texto, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(frame_anotado, (x1, y1 - h - 10), (x1 + w, y1), det['color'], -1)
            
            # Texto
            cv2.putText(frame_anotado, texto, (x1, y1 - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return frame_anotado

# Crear instancia del sistema híbrido
# Si tienes un modelo personalizado, cambia None por la ruta al modelo
sistema_hibrido = SistemaDeteccionHibrido(
    modelo_coco_path='yolov8n.pt',
    modelo_armas_path=None  # Cambia por: 'ruta/a/tu/modelo_armas.pt'
)

print("\n✓ Sistema Híbrido configurado y listo")

🔧 Inicializando Sistema Híbrido de Detección...
✓ Modelo COCO cargado: yolov8n.pt
⚠️ Modelo de armas no disponible (usar ruta válida)

📋 Configuración del sistema:
   - Modelos activos: 1
   - Clases COCO: 4

✓ Sistema Híbrido configurado y listo


In [16]:
# 🎥 SISTEMA DE DETECCIÓN EN TIEMPO REAL CON WEBCAM
# Sistema de seguridad para estadios en Ecuador
# Detecta: Botellas, Cuchillos, Bates de baseball, Tijeras

import time
from datetime import datetime

print("="*70)
print("🎥 INICIANDO SISTEMA DE SEGURIDAD PARA ESTADIOS")
print("="*70)

# Verificar que el sistema esté configurado
try:
    # Iniciar cámara
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Cámara no disponible en este entorno")
        print("\n📋 CÓDIGO FUNCIONAL - Copia y ejecuta en tu computadora local:")
        print("\n" + "="*70)
        print("""
import cv2
import time
from datetime import datetime

# Usar el sistema ya configurado
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("🎥 Cámara activada - Presiona 'q' para salir")

frame_count = 0
fps_start = time.time()
alertas = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Calcular FPS
    frame_count += 1
    if frame_count >= 30:
        fps = frame_count / (time.time() - fps_start)
        fps_start = time.time()
        frame_count = 0
    
    # Detectar objetos peligrosos
    detecciones = sistema_actual.detectar(frame, conf_threshold=0.5)
    frame_anotado = sistema_actual.dibujar_detecciones(frame, detecciones)
    
    # Alertas
    if detecciones:
        alertas += len(detecciones)
        cv2.putText(frame_anotado, f"⚠️ ALERTA: {len(detecciones)} OBJETO(S) PELIGROSO(S)", 
                   (50, 50), cv2.FONT_HERSHEY_BOLD, 1, (0, 0, 255), 3)
        
        for det in detecciones:
            print(f"🚨 {det['alerta']} - {det['confianza']:.0%}")
    
    # Info en pantalla
    h = frame_anotado.shape[0]
    cv2.putText(frame_anotado, f"FPS: {fps if frame_count else 0:.1f}", 
               (10, h-70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(frame_anotado, "🟢 SEGURO" if not detecciones else "🔴 ALERTA", 
               (10, h-40), cv2.FONT_HERSHEY_BOLD, 0.9, 
               (0, 255, 0) if not detecciones else (0, 0, 255), 2)
    cv2.putText(frame_anotado, f"Alertas: {alertas}", 
               (10, h-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Mostrar
    cv2.imshow('Sistema de Seguridad - Estadios Ecuador', frame_anotado)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('s'):
        nombre = f"alerta_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        cv2.imwrite(nombre, frame_anotado)
        print(f"📸 Guardado: {nombre}")

cap.release()
cv2.destroyAllWindows()
print(f"\\n✅ Sistema detenido - Total alertas: {alertas}")
        """)
        print("="*70)
        print("\n✅ El código está listo para copiar y ejecutar")
        print("📌 Ejecuta este código en tu computadora local con cámara")
        
    else:
        print("✅ Cámara detectada - Iniciando detección...")
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        print("\n⌨️ CONTROLES:")
        print("   'q' → Salir")
        print("   's' → Guardar captura")
        print("="*70)
        
        frame_count = 0
        fps_start = time.time()
        alertas = 0
        fps = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            if frame_count >= 30:
                fps = frame_count / (time.time() - fps_start)
                fps_start = time.time()
                frame_count = 0
            
            detecciones = sistema_actual.detectar(frame, conf_threshold=0.5)
            frame_anotado = sistema_actual.dibujar_detecciones(frame, detecciones)
            
            if detecciones:
                alertas += len(detecciones)
                cv2.putText(frame_anotado, f"⚠️ {len(detecciones)} OBJETO(S) PELIGROSO(S)", 
                           (50, 50), cv2.FONT_HERSHEY_BOLD, 1, (0, 0, 255), 3)
                for det in detecciones:
                    print(f"🚨 [{datetime.now().strftime('%H:%M:%S')}] {det['alerta']} - {det['confianza']:.0%}")
            
            h = frame_anotado.shape[0]
            cv2.putText(frame_anotado, f"FPS: {fps:.1f}", (10, h-70), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame_anotado, "🟢 SEGURO" if not detecciones else "🔴 ALERTA", 
                       (10, h-40), cv2.FONT_HERSHEY_BOLD, 0.9, 
                       (0, 255, 0) if not detecciones else (0, 0, 255), 2)
            cv2.putText(frame_anotado, f"Alertas: {alertas}", (10, h-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            cv2.imshow('Sistema de Seguridad - Estadios Ecuador', frame_anotado)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):
                nombre = f"alerta_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
                cv2.imwrite(nombre, frame_anotado)
                print(f"📸 Guardado: {nombre}")
        
        cap.release()
        cv2.destroyAllWindows()
        print(f"\n✅ Sistema detenido - Total alertas: {alertas}")

except Exception as e:
    print(f"❌ Error: {e}")
    print("\n💡 El sistema necesita ejecutarse en un entorno con cámara física")

🎥 INICIANDO SISTEMA DE SEGURIDAD PARA ESTADIOS
❌ Cámara no disponible en este entorno

📋 CÓDIGO FUNCIONAL - Copia y ejecuta en tu computadora local:


import cv2
import time
from datetime import datetime

# Usar el sistema ya configurado
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("🎥 Cámara activada - Presiona 'q' para salir")

frame_count = 0
fps_start = time.time()
alertas = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Calcular FPS
    frame_count += 1
    if frame_count >= 30:
        fps = frame_count / (time.time() - fps_start)
        fps_start = time.time()
        frame_count = 0
    
    # Detectar objetos peligrosos
    detecciones = sistema_actual.detectar(frame, conf_threshold=0.5)
    frame_anotado = sistema_actual.dibujar_detecciones(frame, detecciones)
    
    # Alertas
    if detecciones:
        alertas += len(detecciones)
        cv2.putText(frame_anotado, f"⚠️